# Исследование корреляции солнечной активности и биржи

In [1]:
import pandas as pd

In [2]:
#url = 'https://www.spaceweather.gc.ca/solar_flux_data/daily_flux_values/fluxtable.txt'
df = pd.read_csv('fluxtable.txt', delim_whitespace=True, skiprows=[1])

# Объединяем fluxdate и fluxtime в одно поле timestamp
df['timestamp'] = pd.to_datetime(df['fluxdate'].astype(str) + df['fluxtime'].astype(str), format='%Y%m%d%H%M%S')

# Удаляем старые колонки fluxdate и fluxtime
df.drop(columns=['fluxdate', 'fluxtime'], inplace=True)

# Перемещаем колонку timestamp в начало
df = df[['timestamp'] + [col for col in df.columns if col != 'timestamp']]
df.head(3)

,timestamp,fluxjulian,fluxcarrington,fluxobsflux,fluxadjflux,fluxursi
0,2004-10-28 17:00:00,2453307.229,2022.605,132.7,130.9,117.8
1,2004-10-28 20:00:00,2453307.354,2022.610,135.8,134.0,120.6
2,2004-10-29 20:00:00,2453308.354,2022.646,130.6,128.8,115.9


In [3]:
solar=df.set_index('timestamp')[['fluxadjflux']].resample('D').mean().reset_index()
solar.head(3)

,timestamp,fluxadjflux
0,2004-10-28,132.450000
1,2004-10-29,130.550000
2,2004-10-30,139.466667


In [4]:
import yfinance as yf

In [5]:
tickers = '''AAPL, MSFT, AMZN, GOOGL, META, TSLA, JNJ, JPM, V, PG, NVDA, WMT, 
             DIS, BAC, MA, INTC, PFE, XOM, NFLX, ADBE, CRM, KO, PEP, GE'''

# Загружаем данные с конкретной начальной даты
raw_data = yf.download(
    tickers=tickers,
    start="2004-10-28",
    interval="1d",
    progress=False,
    group_by='ticker',
    threads=True  # Параллельная загрузка для ускорения
)

YF.download() has changed argument auto_adjust default to True


In [6]:
raw_data.head(3)

Ticker           ADBE                                             MA            
Price            Open       High        Low      Close   Volume Open High Low   
Date                                                                            
2004-10-28  27.638787  27.938666  27.528833  27.733749  5088800  NaN  NaN NaN  \
2004-10-29  27.673771  28.008636  27.368895  28.003637  5136000  NaN  NaN NaN   
2004-11-01  28.103597  28.483444  27.893683  28.158575  4982600  NaN  NaN NaN   

Ticker                   ... META                              JNJ              
Price      Close Volume  ... Open High Low Close Volume       Open       High   
Date                     ...                                                    
2004-10-28   NaN    NaN  ...  NaN  NaN NaN   NaN    NaN  32.420731  32.707941  \
2004-10-29   NaN    NaN  ...  NaN  NaN NaN   NaN    NaN  32.623461  32.933195   
2004-11-01   NaN    NaN  ...  NaN  NaN NaN   NaN    NaN  32.859991  33.164094   

Ticker                                     
Price             Low      Close   Volume  
Date                                       
2004-10-28  32.375680  32.668518  5581200  
2004-10-29  32.612198  32.876881  6801500  
2004-11-01  32.719203  32.910675  6724200  

[3 rows x 120 columns]

In [7]:
# Извлекаем цены закрытия и преобразуем структуру
close_prices = (
    raw_data
    .xs('Close', level=1, axis=1)      # Выбираем уровень Close
    .sort_index()                       # Сортируем по времени
)
close_prices.head(3)

Ticker,ADBE,MA,NFLX,PFE,CRM,TSLA,PG,AMZN,AAPL,XOM,...,V,GE,MSFT,DIS,INTC,NVDA,JPM,WMT,META,JNJ
Date,,,,,,,,,,,,,,,,,,,,,
2004-10-28,27.733749,NaN,1.370000,11.836782,5.009520,NaN,29.297028,1.7225,0.785292,24.671705,...,NaN,97.818985,17.440014,19.968208,12.849727,0.112572,22.336199,11.939093,NaN,32.668518
2004-10-29,28.003637,NaN,1.352857,11.935734,5.051784,NaN,29.222805,1.7065,0.788452,24.981302,...,NaN,98.077690,17.415115,20.047699,12.843953,0.110585,22.405857,11.923610,NaN,32.876881
2004-11-01,28.158575,NaN,1.347143,11.873885,5.394868,NaN,29.120024,1.7550,0.789205,24.778278,...,NaN,97.876488,17.483599,20.055651,12.947815,0.112955,22.347799,11.908138,NaN,32.910675


In [8]:
# Для солнечных данных
solar = df.set_index('timestamp')[['fluxadjflux']].resample('D').mean()

# Для цен акций (ваш текущий код)
close_prices = raw_data.xs('Close', level=1, axis=1).sort_index()

# Объединяем данные в один DataFrame
combined = solar.join(close_prices, how='inner').dropna()


combined.head(5)

,fluxadjflux,ADBE,MA,NFLX,PFE,CRM,TSLA,PG,AMZN,AAPL,...,V,GE,MSFT,DIS,INTC,NVDA,JPM,WMT,META,JNJ
2012-05-18,134.033333,31.309999,36.651836,9.994286,13.038117,36.192844,1.837333,44.061729,10.6925,15.961040,...,25.710617,70.976334,23.397007,38.746361,18.292826,0.276960,23.570280,16.045568,38.084522,44.162270
2012-05-21,128.333333,32.009998,38.139969,10.248571,13.020788,37.055531,1.918000,43.971554,10.9055,16.890930,...,26.527771,71.613060,23.780699,39.259319,18.348955,0.281774,22.880556,16.202351,33.900501,44.245934
2012-05-22,123.900000,32.009998,38.405647,9.672857,12.922583,37.154976,2.053333,43.805065,10.7665,16.761227,...,27.023081,71.837784,23.788689,39.259319,18.264750,0.278335,23.936255,16.379683,30.882034,44.280788
2012-05-23,119.966667,32.180000,38.744690,10.272857,12.760837,37.443363,2.068000,43.277878,10.8640,17.170202,...,27.201120,71.837784,23.269117,39.100113,17.850763,0.285213,24.112215,16.598146,31.878227,44.106510
2012-05-24,119.766667,31.540001,38.999233,10.038571,12.789718,36.349472,2.018667,43.402737,10.7620,17.012516,...,27.338072,72.099983,23.237135,39.303532,17.998114,0.277648,23.908104,16.724092,32.904305,44.416225


In [9]:
# Рассчитываем корреляции с статистической значимостью
from scipy.stats import pearsonr
ticker_columns = [col for col in combined.columns if col != 'fluxadjflux']

corr_list = []
for ticker in ticker_columns:
    r, p_value = pearsonr(combined['fluxadjflux'], combined[ticker])
    corr_list.append({
        'Ticker': ticker,
        'Correlation': r,
        'p-value': p_value,
        'Significant (5%)': p_value < 0.05
    })

corr_df = pd.DataFrame(corr_list).sort_values('Correlation', key=abs, ascending=False)

print("Детализированные результаты корреляции:")
print(corr_df)

Детализированные результаты корреляции:
   Ticker  Correlation        p-value  Significant (5%)
9     XOM     0.703125   0.000000e+00              True
19   NVDA     0.595199  1.881381e-306              True
15     GE     0.480626  6.820792e-185              True
18   INTC    -0.466321  9.739918e-173              True
8    AAPL     0.450733  3.889767e-160              True
21    WMT     0.438294  1.531870e-150              True
16   MSFT     0.415934  2.851852e-134              True
12  GOOGL     0.371646  1.691134e-105              True
11     KO     0.368140  2.094481e-103              True
17    DIS    -0.363325  1.425641e-100              True
6      PG     0.360829   4.010110e-99              True
5    TSLA     0.356313   1.559911e-96              True
20    JPM     0.341031   4.446096e-88              True
22   META     0.308040   2.183026e-71              True
13    PEP     0.302414   9.564238e-69              True
1      MA     0.288541   1.764258e-62              True
14      

Если эффект есть, значит он должен быть устойчивым.
Просто разделим данные на два промежутка. Выберем дату произвольно

In [10]:
# Разделяем данные на две части по дате
split_date = '2019-01-01'

combined_before = combined.loc[:split_date]  # Все данные ДО 2019-01-01
combined_after = combined.loc[split_date:]   # Все данные ПОСЛЕ 2019-01-01 (включительно)

# Создаем функцию для расчета корреляций
def calculate_correlations(df):
    corr_list = []
    for ticker in df.columns.drop('fluxadjflux'):
        r, p_value = pearsonr(df['fluxadjflux'], df[ticker])
        corr_list.append({
            'Ticker': ticker,
            'Correlation': r,
            'p-value': p_value,
            'Significant (5%)': p_value < 0.05
        })
    return pd.DataFrame(corr_list).sort_values('Correlation', key=abs, ascending=False)

# Считаем для первого периода
print("Корреляции ДО 2019-01-01:")
corr_before = calculate_correlations(combined_before)
print(corr_before)

# Считаем для второго периода
print("\nКорреляции ПОСЛЕ 2019-01-01:")
corr_after = calculate_correlations(combined_after)
print(corr_after)

Корреляции ДО 2019-01-01:
   Ticker  Correlation        p-value  Significant (5%)
7    AMZN    -0.708256  1.286142e-253              True
12  GOOGL    -0.704263  1.490076e-249              True
22   META    -0.693574  5.283452e-239              True
20    JPM    -0.690082  1.175245e-235              True
14      V    -0.688774  2.046654e-234              True
16   MSFT    -0.687889  1.405159e-233              True
19   NVDA    -0.687399  4.062678e-233              True
13    PEP    -0.678201  1.282399e-224              True
11     KO    -0.671477  1.340083e-218              True
0    ADBE    -0.667547  3.727956e-215              True
23    JNJ    -0.660193  7.536710e-209              True
2    NFLX    -0.654410  5.151881e-204              True
4     CRM    -0.649794  3.127279e-200              True
1      MA    -0.644770  3.457723e-196              True
8    AAPL    -0.637069  3.892665e-190              True
18   INTC    -0.630538  3.880576e-185              True
10    BAC    -0.615966

Эффект не повторился.
